In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from graphics import simple_axes, adjust_raster_ticks, add_scalebar, hide_axes

In [ ]:
base = Path("../build")
iv_stats = pd.read_csv(base / "iv_stats.csv", index_col=["cell", "epoch", "sweep"])
sweep_stats = pd.read_csv(base / "sweep_stats.csv", index_col=["cell", "epoch", "sweep"])
epoch_stats = pd.read_csv(base / "epoch_stats.csv", index_col=["cell", "epoch"])

In [ ]:
cell = "2d21b861"
epoch_idx = [3, 4, 5, 7, 8]
epochs = epoch_stats.loc[cell].loc[epoch_idx]
sweeps = sweep_stats.loc[cell].loc[epoch_idx]
steps = iv_stats.loc[cell].loc[epoch_idx]

In [ ]:
bin_size = 10
floor = np.floor(steps.current.min() / bin_size) * bin_size
ceil  = np.ceil(steps.current.max() / bin_size) * bin_size
bins = np.arange(floor, ceil + bin_size, bin_size)
steps_binned = steps.groupby(["epoch", pd.cut(steps.current, bins, labels=False)]).mean()

In [ ]:
fig = plt.figure(figsize=(5, 5))
subfigs = fig.subfigures(1, 2, wspace=0.02)
axes = subfigs[0].subplots(2, sharex=True)
for enumber, epoch in steps_binned.groupby("epoch"):
    axes[0].plot(epoch.current, epoch.voltage, label="%d s" % int(epochs.loc[enumber].time))
axes[0].set_ylabel("V (mV)")
for enumber, epoch in sweeps.groupby("epoch"):
    axes[1].plot(epoch.current, epoch.firing_rate, label="%d s" % int(epochs.loc[enumber].time))
axes[1].set_ylabel("Freq (Hz)")
axes[1].set_xlabel("Current (pA)")
axes[1].legend()
simple_axes(*axes)
subfigs[0].align_ylabels(axes)

marker_style = {"marker": 'o', "linestyle": 'none', "fillstyle": "none"}
axes = subfigs[1].subplots(5, sharex=True, height_ratios=(2, 2, 2, 1, 1))
axes[0].errorbar(epochs.time, epochs.duration_mean, epochs.duration_sd / np.sqrt(epochs.n_spike_sweeps), **marker_style)
axes[0].set_ylim(0, 2.0)
axes[0].set_ylabel("Duration (s)")
axes[1].plot(epochs.time, epochs.slope, **marker_style)
axes[1].set_ylim(0, epochs.slope.max() * 1.1)
axes[1].set_ylabel("f-I slope (Hz/pA)")
axes[2].plot(epochs.time, epochs.rheobase, **marker_style)
axes[2].set_ylim(0, epochs.rheobase.max() * 1.1)
axes[2].set_ylabel("rheobase (pA)")
axes[3].errorbar(epochs.time, epochs.Vm, epochs.Vm_sd / np.sqrt(epochs.n_sweeps), **marker_style)
axes[3].set_ylim(epochs.Vm.mean() - 10, epochs.Vm.mean() + 10)
axes[3].set_ylabel("Vm (mV)")                  
axes[4].errorbar(epochs.time, epochs.Rm, epochs.Rm_sd / np.sqrt(epochs.n_sweeps), **marker_style)
Rmm = epochs.Rm.mean()
axes[4].set_ylim(Rmm * 0.7, Rmm * 1.3)
axes[4].set_ylabel("Rm (mΩ)")
axes[4].set_xlabel("Time (s)")
simple_axes(*axes)
subfigs[1].subplots_adjust(hspace=0.08)
subfigs[1].align_ylabels(axes)

In [ ]:
fig, axes = plt.subplots(6, figsize=(2, 5), sharex=True, height_ratios=(3, 3, 1, 3, 3, 1))
pprox = base / "{}_{:02}.pprox".format(cell, epoch_idx[0])
with open(pprox, "rt") as fp:
    epoch = json.load(fp)
for sweep_idx, pproc in enumerate(epoch["pprox"]):
    axes[0].plot(pproc["events"], [sweep_idx] * len(pproc["events"]), '|')
adjust_raster_ticks(axes[0], gap=2)

pprox = base / "{}_{:02}.pprox".format(cell, epoch_idx[-1])
with open(pprox, "rt") as fp:
    epoch = json.load(fp)
for sweep_idx, pproc in enumerate(epoch["pprox"]):
    axes[3].plot(pproc["events"], [sweep_idx] * len(pproc["events"]), '|')
adjust_raster_ticks(axes[3], gap=2)
hide_axes(*axes)